In [3]:
import os
import random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

import cv2
from ultralytics import YOLO
import torch

# -----------------------
# TensorFlow GPU 설정
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"[INFO] {len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs available")
    except RuntimeError as e:
        print(e)
else:
    print("[INFO] No GPU detected for TensorFlow")

# -----------------------
# Config
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5
}

# Seed 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(CFG['SEED'])

# -----------------------
# 경로 설정
ORIGINAL_TRAIN_DIR = "D:/데이콘 250519 대회/open/train"
FILTERED_TRAIN_DIR = "D:/데이콘 250519 대회/filtered_train"
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

# -----------------------
# Ultralytics YOLO GPU 설정 및 차량 클래스 정의
VEHICLE_CLASSES = [2, 5, 7]  # car, bus, truck 클래스 번호

yolo_model = YOLO('yolov8n.pt')
if torch.cuda.is_available():
    yolo_model.model = yolo_model.model.cuda()
    print("[INFO] YOLO model moved to GPU")
else:
    print("[INFO] YOLO GPU not available, using CPU")

# -----------------------
# 차량 외관 전체 포함 및 보닛/트렁크 열린 이상치 필터링 함수
def is_full_vehicle(detections, img):
    h, w = img.shape[:2]
    img_area = h * w

    for det in detections:
        cls = int(det.cls)
        if cls not in VEHICLE_CLASSES:
            continue

        x1, y1, x2, y2 = det.xyxy[0].cpu().numpy()
        box_w, box_h = x2 - x1, y2 - y1
        area = box_w * box_h
        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
        aspect_ratio = box_w / box_h

        # 차량 중앙 근처 위치 조건 (중앙 30% 영역 내)
        if not (0.35 * w < center_x < 0.65 * w and 0.35 * h < center_y < 0.65 * h):
            continue

        # 차량 박스 크기 최소 비율 조건 (전체 이미지의 35% 이상)
        if area / img_area < 0.35:
            continue

        # 차량 비율 조건 (너무 길거나 너무 좁으면 제거 - 트렁크/보닛 열림 방지)
        if aspect_ratio < 0.9 or aspect_ratio > 2.3:
            continue

        # 보닛/트렁크 열린 차량은 박스 내부 영역 패턴 분석 추가
        # (예: 박스 상단 또는 하단에 지나치게 큰 빈 영역이 있는지 간단 체크)
        # 이미지 영역 crop
        vehicle_crop = img[int(y1):int(y2), int(x1):int(x2)]
        gray = cv2.cvtColor(vehicle_crop, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

        # 상단 10% 영역에 흰색(열린 보닛 등) 픽셀 비율 체크
        top_area = thresh[0:int(0.1 * thresh.shape[0]), :]
        white_ratio_top = np.sum(top_area == 255) / top_area.size

        # 하단 10% 영역 체크 (트렁크 열림 가능성)
        bottom_area = thresh[int(0.9 * thresh.shape[0]):, :]
        white_ratio_bottom = np.sum(bottom_area == 255) / bottom_area.size

        # 너무 흰 영역이 많으면 보닛/트렁크 열린 이상치로 간주
        if white_ratio_top > 0.25 or white_ratio_bottom > 0.25:
            continue

        # 위 조건 모두 통과하면 정상 차량 외관 포함으로 판단
        return True

    return False

# -----------------------
# 차량 외관 이상치 필터링 수행
print("차량 외관 이상치 필터링 시작...")
os.makedirs(FILTERED_TRAIN_DIR, exist_ok=True)
class_dirs = sorted(os.listdir(ORIGINAL_TRAIN_DIR))

filtered_count = 0
total_count = 0

for cls in tqdm(class_dirs, desc="클래스별 필터링"):
    input_dir = os.path.join(ORIGINAL_TRAIN_DIR, cls)
    output_dir = os.path.join(FILTERED_TRAIN_DIR, cls)
    os.makedirs(output_dir, exist_ok=True)

    image_paths = glob(os.path.join(input_dir, "*.jpg"))
    for img_path in image_paths:
        total_count += 1
        img = cv2.imread(img_path)
        results = yolo_model(img, verbose=False)[0]

        if is_full_vehicle(results.boxes, img):
            filename = os.path.basename(img_path)
            cv2.imwrite(os.path.join(output_dir, filename), img)
            filtered_count += 1

print(f"🔍 전체 이미지 수: {total_count}")
print(f"✅ 필터링된 차량 외관 이미지 수: {filtered_count}")
print(f"🧼 이상치 제거 완료. 정제 비율: {filtered_count / total_count:.2%}")

# -----------------------
# 학습 데이터 준비
label_list = sorted(os.listdir(FILTERED_TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

image_paths = glob(os.path.join(FILTERED_TRAIN_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# -----------------------
# 데이터 전처리 함수
def load_and_preprocess(img_path):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img

    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )

    if is_train:
        ds = ds.shuffle(1024)
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds

# -----------------------
# 모델 생성
def build_model(num_classes):
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3),
        weights='imagenet',
        pooling='avg'
    )
    x = layers.Dense(num_classes, activation='softmax')(base.output)
    model = models.Model(inputs=base.input, outputs=x)
    return model

# -----------------------
# Stratified K-Fold 학습
skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1} 시작 ###")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    model = build_model(num_classes=len(label2id))
    model.compile(
        optimizer=optimizers.Adam(CFG['LR']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=CFG['EPOCHS'],
        verbose=1
    )

# -----------------------
# 테스트 데이터 추론 및 제출 파일 생성
test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
test_ds = create_dataset(test_paths, is_train=False)

print("테스트 데이터 추론 시작...")
preds = model.predict(test_ds)
print("추론 완료")

submission = pd.read_csv(SAMPLE_SUB)
for idx, class_name in enumerate(label2id.keys()):
    submission[class_name] = preds[:, idx]

submission.to_csv("submission.csv", index=False)
print("submission.csv 저장 완료")


[INFO] 1 Physical GPUs, 1 Logical GPUs available
[INFO] YOLO GPU not available, using CPU
차량 외관 이상치 필터링 시작...


클래스별 필터링: 100%|██████████| 396/396 [31:13<00:00,  4.73s/it]
c:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


🔍 전체 이미지 수: 33137
✅ 필터링된 차량 외관 이미지 수: 13582
🧼 이상치 제거 완료. 정제 비율: 40.99%

### Fold 1 시작 ###
Epoch 1/15
453/453 [==============================] - 303s 204ms/step - loss: 4.2333 - accuracy: 0.2354 - val_loss: 4.0540 - val_accuracy: 0.2367
Epoch 2/15
453/453 [==============================] - 41s 85ms/step - loss: 1.7778 - accuracy: 0.5934 - val_loss: 2.2768 - val_accuracy: 0.4280
Epoch 3/15
453/453 [==============================] - 40s 85ms/step - loss: 0.7636 - accuracy: 0.8117 - val_loss: 1.4227 - val_accuracy: 0.6091
Epoch 4/15
453/453 [==============================] - 40s 84ms/step - loss: 0.3909 - accuracy: 0.9080 - val_loss: 1.0578 - val_accuracy: 0.7052
Epoch 5/15
453/453 [==============================] - 41s 85ms/step - loss: 0.2412 - accuracy: 0.9412 - val_loss: 0.8703 - val_accuracy: 0.7589
Epoch 6/15
453/453 [==============================] - 41s 86ms/step - loss: 0.1707 - accuracy: 0.9624 - val_loss: 0.7680 - val_accuracy: 0.7943
Epoch 7/15
453/453 [========================